<a href="https://colab.research.google.com/github/safal207/hello-world/blob/master/dl_nlp_yelp_polarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Учим компьютер понимать эмоции в тексте (отзывы Yelp)

**🇷🇺 (RU)**: Давайте научим компьютер понимать, какой отзыв хороший, а какой — плохой! Мы будем использовать настоящие отзывы с сайта Yelp и научим нейронную сеть определять, оставил ли человек положительный или отрицательный отзыв.

---

**🇬🇧 (EN)**: Let's teach a computer to understand if a review is good or bad! We will use real reviews from Yelp and teach a neural network to decide if a person left a positive or negative review.

### 1. Подготовка и настройка

In [ ]:
%tensorflow_version 2.x
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras import utils
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

%matplotlib inline

In [ ]:
# Гиперпараметры
num_words = 10000  # Размер словаря (количество наиболее частых слов)
max_review_len = 100  # Максимальная длина отзыва в словах

### 2. Загрузка данных

**🇷🇺 (RU)**: Сначала нам нужно получить наши данные. Эти команды скачивают большой архив с отзывами из интернета и затем распаковывают его, чтобы мы могли использовать файлы.

---

**🇬🇧 (EN)**: First, we need to get our data. These commands download a big archive with reviews from the internet and then unpack it so we can use the files.

In [ ]:
!wget https://www.dropbox.com/s/ufbhk3kadtnn6h0/yelp_review_polarity_csv.tgz?dl=1 -O yelp_review_polarity_csv.tgz

In [ ]:
!tar -xvf yelp_review_polarity_csv.tgz

#### Просмотр данных
Давайте посмотрим на описание набора данных и примеры из обучающего и тестового файлов.

In [ ]:
!cat yelp_review_polarity_csv/readme.txt

In [ ]:
!head yelp_review_polarity_csv/train.csv

In [ ]:
!wc -l yelp_review_polarity_csv/train.csv
!wc -l yelp_review_polarity_csv/test.csv

### 3. Предобработка данных
Загружаем данные из CSV-файлов в DataFrame-ы pandas.

In [ ]:
train = pd.read_csv(
    "yelp_review_polarity_csv/train.csv", header=None, names=["Class", "Review"]
)

In [ ]:
train.head()

Выделяем данные для обучения: тексты отзывов и их классы (тональность).

In [ ]:
reviews = train["Review"]

In [ ]:
# Классы в датасете: 1 (негативный) и 2 (позитивный). Преобразуем их в 0 и 1.
y_train = train["Class"] - 1

#### Превращаем слова в цифры (Токенизация и Дополнение)

**🇷🇺 (RU)**: Компьютер не понимает слова, но он хорошо работает с числами. Поэтому нам нужно превратить каждое слово в уникальный номер. Этот процесс называется **токенизация**. После этого мы делаем все отзывы одинаковой длины, добавляя «пустые» номера (нули), чтобы нейронная сеть могла с ними работать. Это называется **дополнение** (padding).

---

**🇬🇧 (EN)**: A computer doesn't understand words, but it works well with numbers. So, we need to turn every word into a unique number. This process is called **tokenization**. After that, we make all reviews the same length by adding 'empty' numbers (zeros) so the neural network can work with them. This is called **padding**.

In [ ]:
# Создаем токенизатор Keras, который будет учитывать только 10,000 самых частых слов
tokenizer = Tokenizer(num_words=num_words)

In [ ]:
# Обучаем токенизатор на наших данных
tokenizer.fit_on_texts(reviews)

In [ ]:
# Преобразуем тексты в числовые последовательности
sequences = tokenizer.texts_to_sequences(reviews)

Пример преобразования отзыва в последовательность чисел:

In [ ]:
index = 1
print(reviews[index])
print(sequences[index])

In [ ]:
# Дополняем последовательности до фиксированной длины
x_train = pad_sequences(sequences, maxlen=max_review_len)

### 4. Собираем наш "мозг" (Нейронную сеть)

**🇷🇺 (RU)**: Теперь мы строим «мозг» нашего компьютера. Он состоит из трех частей (слоев):
1.  `Embedding`: Превращает числовые коды слов в осмысленные векторы. Похожие по смыслу слова будут иметь похожие векторы.
2.  `LSTM`: «Память» нашей сети. Она читает слова по порядку и пытается понять общий смысл и контекст всего отзыва.
3.  `Dense`: «Принимающий решение». Этот слой смотрит на то, что поняла LSTM, и делает финальный вывод: отзыв хороший (ближе к 1) или плохой (ближе к 0).

---

**🇬🇧 (EN)**: Now we build the 'brain' of our computer. It has three parts (layers):
1. `Embedding`: Turns the number codes for words into meaningful vectors. Words with similar meanings will have similar vectors.
2. `LSTM`: The 'memory' of our network. It reads the words in order and tries to understand the overall meaning and context of the entire review.
3. `Dense`: The 'decision-maker'. This layer looks at what the LSTM understood and makes the final conclusion: the review is good (closer to 1) or bad (closer to 0).

In [ ]:
model = Sequential()
model.add(Embedding(num_words, 64, input_length=max_review_len))
model.add(LSTM(128))
model.add(Dense(1, activation="sigmoid"))

In [ ]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

**🇷🇺 (RU)**: Теперь мы «учим» наш мозг, показывая ему много примеров. `model.fit` — это команда для начала обучения. Компьютер будет смотреть на отзывы и угадывать, хорошие они или плохие, а потом проверять себя и становиться умнее. График покажет нам, насколько хорошо он учится. После обучения мы проверим его на совершенно новых отзывах, чтобы увидеть, насколько хорошо он на самом деле научился.

---

**🇬🇧 (EN)**: Now we 'teach' our brain by showing it lots of examples. `model.fit` is the command to start the training. The computer will look at reviews, guess if they are good or bad, then check its answer and get smarter. The graph will show us how well it's learning. After training, we will test it on brand new reviews to see how well it really learned.

In [ ]:
model.load_weights(model_save_path)

In [ ]:
test = pd.read_csv(
    "yelp_review_polarity_csv/test.csv", header=None, names=["Class", "Review"]
)

In [ ]:
test_sequences = tokenizer.texts_to_sequences(test["Review"])

In [ ]:
x_test = pad_sequences(test_sequences, maxlen=max_review_len)

In [ ]:
y_test = test["Class"] - 1

In [ ]:
model.evaluate(x_test, y_test, verbose=1)

### 6. Прогнозирование на новом отзыве

In [ ]:
text = '''The SmartBurger restaurant is awful. It’s a small shabby place. 
The food is really bad and very expensive.  The host and waiters are rude. 
I will never visit the SmartBurger again!
'''

In [ ]:
sequence = tokenizer.texts_to_sequences([text])

In [ ]:
data = pad_sequences(sequence, maxlen=max_review_len)

In [ ]:
result = model.predict(data)

In [ ]:
if result[[0]] < 0.5:
    print("Отзыв отрицательный")
else:
    print("Отзыв положительный")

### 7. Анализ ошибок модели

Чтобы лучше понять, где модель ошибается, давайте посмотрим на несколько отзывов, которые были классифицированы неверно.

In [ ]:
# Получаем предсказания для всего тестового набора
predictions = (model.predict(x_test) > 0.5).astype("int32")

# Находим индексы, где предсказания не совпадают с истинными метками
y_test_flat = y_test.to_numpy()
misclassified_indices = np.where(predictions.flatten() != y_test_flat)[0]

print(f"Количество неверно классифицированных отзывов: {len(misclassified_indices)}")

# Выводим несколько примеров
print("\nПримеры неверно классифицированных отзывов:\n")
for i, index in enumerate(misclassified_indices[:5]):
    review_text = test.iloc[index]['Review']
    true_label = "Позитивный" if y_test.iloc[index] == 1 else "Негативный"
    predicted_label = "Позитивный" if predictions[index][0] == 1 else "Негативный"
    
    print(f"--- Пример {i+1} ---")
    print(f"Текст: {review_text[:500]}...")
    print(f"Истинная метка: {true_label}")
    print(f"Предсказанная метка: {predicted_label}")
    print("-" * (15 + len(str(i+1))))

## Задания для самостоятельной работы

1. Используйте сеть GRU вместо LSTM для определения тональности отзывов Yelp. Сравните скорость обучения и качество работы обученной сети.
2. Меняйте гиперпараметры нейросети, чтобы повысить качество работы:
  - Длину вектора представления слов в слое Embedding.
  - Количество нейронов на рекуррентном слое (LSTM или GRU).
  - Количество рекуррентные слоев.
  - Тип оптимизатора (`adam`, `rmsprop` и др.)
  - Количество эпох обучения.
  - Разрмер мини-выборки.
3.Попробуйте применить вместо токенизатора Keras более совершенные:
  - [spaCy Tokenizer](https://spacy.io/usage/spacy-101#annotations-token)
  - [Huggingface Tokenizers](https://github.com/huggingface/tokenizers)
  
    Эти токенизаторы сложнее в использовании, но работают лучше.
4. Переделайте нейросеть для распознавания набора данных [Yelp reviews - Full](https://s3.amazonaws.com/fast-ai-nlp/yelp_review_full_csv.tgz), в котором 4 класса по количеству звезд у отзывов.